In [ ]:
import numpy as np
from collections import Counter
import math

# Training data
documents = {
    'd1': (['free', 'free', 'free', 'buy', 'discount', 'combo', 'pleasure'], 'S'),
    'd2': (['free', 'free', 'free', 'discount', 'pleasure', 'smile', 'smile', 'smile'], 'S'),
    'd3': (['cat', 'mouse'], 'N'),
    'd4': (['cat', 'cat', 'dog', 'dog', 'dog', 'dog'], 'N'),
    'd5': (['mouse'], 'N'),
}

# Test data
test_documents = {
    'd6': ['dog', 'cat', 'mouse', 'cat'],
    'd7': ['free', 'free', 'smile'],
}

# Step 1: Compute Mutual Information (MI)
def compute_mi(word, documents):
    N = len(documents)
    word_in_class = Counter()
    class_counter = Counter()

    for doc, (words, cls) in documents.items():
        class_counter[cls] += 1
        if word in words:
            word_in_class[(word, cls)] += 1

    mi = 0
    for cls in class_counter:
        p_w_c = word_in_class[(word, cls)] / N
        p_w = sum(word_in_class[(word, c)] for c in class_counter) / N
        p_c = class_counter[cls] / N
        if p_w_c > 0:
            mi += p_w_c * math.log(p_w_c / (p_w * p_c), 2)

    return mi

# Collect all words
all_words = set(word for doc in documents for word in documents[doc][0])

# Compute MI for each word
mi_scores = {word: compute_mi(word, documents) for word in all_words}

# Sort MI scores
sorted_mi_scores = sorted(mi_scores.items(), key=lambda item: item[1], reverse=True)

# Select features with the highest MI score
highest_mi_score = sorted_mi_scores[0][1]
top_features = [word for word, score in sorted_mi_scores if score == highest_mi_score]

# Find the second highest MI score
second_highest_mi_score = None
for score in sorted_mi_scores:
    if score[1] < highest_mi_score:
        second_highest_mi_score = score[1]
        break

# Select features with the second highest MI score
second_top_features = [word for word, score in sorted_mi_scores if score == second_highest_mi_score]

# Combine the top features
top_features.extend(second_top_features)

print(f"Mutual Information Scores: {mi_scores}")
print(f"Top features selected: {top_features}")

# Step 2: Compute TF*IDF scores
def compute_tf(word, document):
    return document.count(word) / len(document)

def compute_idf(word, documents):
    N = len(documents)
    df = sum(1 for doc in documents if word in documents[doc][0])
    return math.log(N / (1 + df))

def compute_tfidf(word, document, documents):
    tf = compute_tf(word, document)
    idf = compute_idf(word, documents)
    return tf * idf

# Step 3: Represent each document with TF*IDF values of the selected features
tfidf_matrix = []

for doc in documents:
    tfidf_vector = [compute_tfidf(word, documents[doc][0], documents) for word in top_features]
    tfidf_matrix.append(tfidf_vector)

tfidf_matrix = np.array(tfidf_matrix)
print(f"TF*IDF Matrix:\n{tfidf_matrix}")

# Step 4: Compute TF*IDF for d6
d6_tfidf = [compute_tfidf(word, test_documents['d6'], documents) for word in top_features]
d6_tfidf = np.array(d6_tfidf)
print(f"TF*IDF values for d6: {d6_tfidf}")

# Step 5: Compute TF*IDF for d7
d7_tfidf = [compute_tfidf(word, test_documents['d7'], documents) for word in top_features]
d7_tfidf = np.array(d7_tfidf)
print(f"TF*IDF values for d7: {d7_tfidf}")

# Step 6: Predict the class label for d6 using KNN
def euclidean_distance(v1, v2):
    return np.sqrt(np.sum((v1 - v2) ** 2))

def knn_classify(tfidf_matrix, new_vector, k=1):
    distances = []
    for i, vec in enumerate(tfidf_matrix):
        dist = euclidean_distance(vec, new_vector)
        distances.append((dist, i))
    distances.sort()
    nearest_neighbors = distances[:k]

    class_votes = Counter()
    for _, i in nearest_neighbors:
        class_votes[documents[f'd{i+1}'][1]] += 1

    return class_votes.most_common(1)[0][0]

d6_class = knn_classify(tfidf_matrix, d6_tfidf)
d7_class = knn_classify(tfidf_matrix, d7_tfidf)

print(f'd6 is classified as: {d6_class}')
print(f'd7 is classified as: {d7_class}')


Mutual Information Scores: {'buy': 0.2643856189774724, 'dog': 0.14739311883324124, 'smile': 0.2643856189774724, 'cat': 0.2947862376664825, 'discount': 0.5287712379549449, 'mouse': 0.2947862376664825, 'combo': 0.2643856189774724, 'pleasure': 0.5287712379549449, 'free': 0.5287712379549449}
Top features selected: ['discount', 'pleasure', 'free', 'cat', 'mouse']
TF*IDF Matrix:
[[0.07297509 0.07297509 0.21892527 0.         0.        ]
 [0.0638532  0.0638532  0.19155961 0.         0.        ]
 [0.         0.         0.         0.25541281 0.25541281]
 [0.         0.         0.         0.17027521 0.        ]
 [0.         0.         0.         0.         0.51082562]]
TF*IDF values for d6: [0.         0.         0.         0.25541281 0.12770641]
TF*IDF values for d7: [0.         0.         0.34055042 0.         0.        ]
d6 is classified as: N
d7 is classified as: S
